In [4]:
!apt-get install openjdk-21-jre-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-21-openjdk-amd64/bin/java
!java -version

openjdk version "21.0.5" 2024-10-15
OpenJDK Runtime Environment (build 21.0.5+11-Ubuntu-1ubuntu122.04)
OpenJDK 64-Bit Server VM (build 21.0.5+11-Ubuntu-1ubuntu122.04, mixed mode, sharing)


In [21]:
import pyserini
from pyserini.search.lucene import LuceneSearcher
from pyserini.index import LuceneIndexReader
from IPython.core.display import display, HTML
from pyserini.search import get_topics
import heapq

In [11]:
def initialize_model(index_path):
  searcher = LuceneSearcher.from_prebuilt_index(index_path)
  reader = LuceneIndexReader.from_prebuilt_index(index_path)
  return searcher, reader

In [31]:
MAX_RECS = 5

def get_recs(query: str, n: int, searcher: LuceneSearcher, reader: LuceneIndexReader) -> list[str]:
  hits = searcher.search(query, n)
  for i in hits:
    doc_vec = reader.get_document_vector(i.docid)
    h = []
    for t, f in doc_vec.items():
      tf = f / len(doc_vec)
      df = reader.get_term_counts(t)[0]
      if df == 0:
        idf = 0
      else:
        idf = reader.stats()['documents'] / df
      tf_idf = tf * idf
      heapq.heappush(h, (tf_idf, t))
      if len(h) > MAX_RECS:
        heapq.heappop(h)
  return [heapq.heappop(h)[1] for _ in range(len(h))]

In [33]:
query = "Are there black bear attacks there?"

searcher, reader = initialize_model("robust04")
reader.get_term_counts("be")
get_recs(query, 10, searcher, reader)

JavaException: JVM exception occurred: Index 0 out of bounds for length 0 java.lang.IndexOutOfBoundsException